# FY4A AGRI L1 Reader Introduction

## 1. Support Files

FY4A AGRI data in NetCDF format.

Both Full DISK and regional images are supported.

Example filenames:

Full DISK:
    
    FY4A-_AGRI--_N_DISK_1047E_L1-_FDI-_MULT_NOM_20190807060000_20190807061459_4000M_V0001.HDF

REGC:

    FY4A-_AGRI--_N_REGC_1047E_L1-_FDI-_MULT_NOM_20190807045334_20190807045750_1000M_V0001.HDF

*Full disk scans are identified by DISK , regional scans by REGC.*

Data Links:

    Real Time Data Service (30 days) and Introduction Files:
        https://fy4.nsmc.org.cn/data/en/data/realtime.html
    History data (2018-03-12 -- ):
        http://satellite.nsmc.org.cn/PortalSite/Data/Satellite.aspx
    FY4A official weather application platform:
        http://rsapp.nsmc.org.cn/geofy/

## 2. Calibration

You have three options:

1. The raw detector counts (All channels)

2. Reflectance (C01 - C06)

3. Radiance and Brightness Temperature (C07 - C14)

## 3. Examples

### Loading data

In [1]:
import os, glob
from satpy.scene import Scene
from satpy.utils import debug_on
from trollimage.colormap import greys, spectral

In [2]:
# load FY4A filenames
filenames = glob.glob('/xin/data/FY4A/20190807/FY4A-_AGRI*4000M_V0001.HDF')

# create the scene object
scn = Scene(filenames, reader='agri_l1')

# check available channels
scn.available_dataset_names()

['C01',
 'C02',
 'C03',
 'C04',
 'C05',
 'C06',
 'C07',
 'C08',
 'C09',
 'C10',
 'C11',
 'C12',
 'C13',
 'C14',
 'satellite_azimuth_angle',
 'satellite_zenith_angle',
 'solar_azimuth_angle',
 'solar_glint_angle',
 'solar_zenith_angle']

In [3]:
# take the ir channel as example
ir_channel = 'C12'
scn.load([ir_channel], generate=False, calibration='brightness_temperature')

### Generate the full disk image

In [4]:
# display in notebook
scn.show(ir_channel)

<img src='./figures/agri_C12.png'>

In [5]:
# save to file
# scn.save_dataset(ir_channel, filename='{sensor}_{name}.png')

### Generate the full disk image (True color)

In [6]:
# get a list of all available composites for the current scene
scn.available_composite_names()

Too many possible datasets to load for 3.9
Too many possible datasets to load for 3.9
Too many possible datasets to load for 3.9
Too many possible datasets to load for DatasetID(name=None, wavelength=3.9, resolution=None, polarization=None, calibration=None, level=None, modifiers=())
Too many possible datasets to load for 3.9
Too many possible datasets to load for 3.9
Too many possible datasets to load for DatasetID(name=None, wavelength=3.9, resolution=None, polarization=None, calibration=None, level=None, modifiers=())


['ash',
 'dust',
 'fog',
 'green',
 'green_snow',
 'ir108_3d',
 'ir_cloud_day',
 'natural_color',
 'natural_color_sun',
 'night_background',
 'night_background_hires',
 'overview',
 'overview_sun',
 'true_color']

In [7]:
#Beware that this step might need much memory available on the processing machine (depending on the number of cpu cores)

composite = 'true_color'
scn.load([composite])
scn.show(composite)
# scn.save_dataset(composite, filename='{sensor}_{name}.png')

Required file type 'agri_l1_4000m_geo' not found or loaded for 'satellite_zenith_angle'
Required file type 'agri_l1_4000m_geo' not found or loaded for 'satellite_azimuth_angle'
Required file type 'agri_l1_4000m_geo' not found or loaded for 'solar_azimuth_angle'
Required file type 'agri_l1_4000m_geo' not found or loaded for 'solar_zenith_angle'


<img src='./figures/agri_true_color.png'>

### Generate the composite on a Region of Interest

I take the typhoon LEKIMA as an example.

We can define a map-projection and a sub area, and project the data on this area.

`Pyresample` can be used to define the area easily.

This definition can also be put in the `area.yaml` configuration file.

In [8]:
from pyresample import get_area_def

area_id = 'lekima'

x_size = 549
y_size = 499
area_extent = (-1098006.560556, -967317.140452, 1098006.560556, 1026777.426728)
projection = '+proj=laea +lat_0=19.0 +lon_0=128.0 +ellps=WGS84'
description = "Typhoon Lekima"
proj_id = 'laea_128.0_19.0'

areadef = get_area_def(area_id, description, proj_id, projection,x_size, y_size, area_extent)

You can generate the area easily by [coord2area_def.py](https://github.com/pytroll/satpy/blob/master/utils/coord2area_def.py)

Here's the output of `python coord2area_def.py lekima_4km laea 10 28 118 138 4`:

```
lekima_4km:
  description: lekima_4km
  projection:
    proj: laea
    ellps: WGS84
    lat_0: 19.0
    lon_0: 128.0
  shape:
    height: 499
    width: 549
  area_extent:
    lower_left_xy: [-1098006.560556, -967317.140452]
    upper_right_xy: [1098006.560556, 1026777.426728]
```

Now, you can add the configuration to `$PPP_CONFIG_DIR/areas.yaml` and use it directly

In [9]:
# If you have added it to areas.yaml, you can use the name directly:
os.environ['PPP_CONFIG_DIR'] = '/yin_raid/xin/satpy_config/'
lekima_scene = scn.resample('lekima_4km')

# Otherwise, you need to use the areadef defined above:
# lekima_scene = scn.resample(areadef)

In [10]:
lekima_scene.show(composite)
# lekima_scene.save_dataset(composite, filename='{sensor}_{name}_resampled.png')

<img src='./figures/agri_true_color_resampled.png'>

If you want to generate pictures with specific colormap like the figure below,

please check another notebook about `enhancement`.

<img src='./figures/agri_C12_resampled_colorize.png'>